# Rossmann Store Sales Forecasting
**Business Analytics (Spring, 2017)**  
서한빈(석사2) @ Data Science, Seoultech  

Submit Date: 2017.3.15. (Assignment 02)

* * *  
## 0.1. Problem
### 유럽의 올리브영이라는 드럭스토어 Rossmann 매출 예측 문제
지난 과제에서 데이터를 탐색하고, 매출을 예측하는 문제를 해결해 ~~보았다.~~(보려고 했다.)
이번 과제에서는 다시 데이터를 탐색하는 과정을 거쳐 아래 2가지 유형의 변수를 어떻게 활용해 볼 수 있을지 고민해 보는 시간을 가져보려 한다.

### Assignment's Problem
1. How to use CompetitionOpenSince[Month/Year]
2. How to use Promo2Since[Year/Week]
> Explain your ideas
3. Summarize answers using a word processor

* * *  
## 0.2. 데이터 호출, 탐색, 정제

먼저 train 데이터에 대해 정제를 해보자.

#### 데이터 호출


In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np

f_dir = "D:/course_2017_spring/BA/Data/W1_Rossmann/"
raw_train = pd.read_csv(f_dir+"train.csv")
raw_store = pd.read_csv(f_dir+"store.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 데이터 탐색, 정제(train data)
수업에서 확인했다 싶이, `StateHoliday` 변수에는 표현상 오류가 존재했다.  
0을 표현하는데 있어, `str` 0과 `int` 0의 차이가 있어, 이를 모두 `str` 타입으로 통일하였다.

In [2]:
print("raw_train.dtypes :\n", raw_train.dtypes)
raw_train.head()

raw_train.dtypes :
 Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [3]:
print(raw_train['StateHoliday'].value_counts())
raw_train.loc[raw_train.StateHoliday == 0, 'StateHoliday'] = '0'
print(raw_train['StateHoliday'].value_counts())

0    855087
0    131072
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64
0    986159
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64


그 다음.  
`Open` 변수가 1인것만 의미가 있다는 것을 지난 과제에서 확인했다.  
지워버리고 `df_train` 이라는 테이블을 새로 생성했다. 왜냐면 `raw_train`은 다음에 또 쓸 꺼 같아서 새로 만드는 테이블로 작업을 하려고 하기 떄문이다.

그리고 각각의 변수가 몇개의 값을 갖고 있는지 확인했다. 
명목변수 중 `binary`인 변수는 duumy coding을 할 필요가 없기 떄문이다.

In [4]:
df_train = raw_train[raw_train['Open']==1]
del df_train['Open']

for key in df_train.keys() :
    print(key, "\t", len(set(df_train[key])))

Store 	 1115
DayOfWeek 	 7
Date 	 942
Sales 	 21734
Customers 	 4086
Promo 	 2
StateHoliday 	 4
SchoolHoliday 	 2


흠...  
보니까 대충 Promo, SchoolHoliday 변수는 `binary` 값을 갖는 변수 일 것 같다.  
나머지 명목변수 DayOfWeek, StateHoliday 변수는 dummy coding이 필요한 부분이라 판단을 하기 충분하다고 생각된다.

In [5]:
dm_DayOfWeek = pd.get_dummies(df_train['DayOfWeek'], prefix='DayOfWeek')
dm_StateHoliday = pd.get_dummies(df_train['StateHoliday'], prefix='StateHoliday')
df_train = df_train.drop(['DayOfWeek', 'StateHoliday'], axis=1)

df_train[list(set(dm_DayOfWeek))] = dm_DayOfWeek
df_train[list(set(dm_StateHoliday))] = dm_StateHoliday
del df_train['Customers']

이때, `Customers` 변수를 제외하는 이유가 있다.  
매출을 예측하는건 '미래' 일인데, 방문고객수를 미리 알고 있다는 건 말이 안 되기 때문에 예측에서 사용할 수 없는 변수이다.

#### 데이터 탐색, 정제(store data)
이제 train data는 적당히 본 거 같고, store data를 살펴보자.

In [6]:
print("raw_train.dtypes :\n", raw_store.dtypes)
raw_store.head(10)

raw_train.dtypes :
 Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN


흠...  
여기서 적당히 당장 써먹을 수 있는 변수는 StoreType과 Assortment라는 명목변수 밖에 보이지 않는다.  
~~(과제를 수행하기 위한 큰 그림)~~  
이걸 dummy coding해서 DataFrame 형식으로 만들고 table merge 해야겠다..!  
그래서 `merge_table` 이라는 테이블을 생성했다.

In [7]:
dm_StoreType = pd.get_dummies(raw_store['StoreType'], prefix='StoreType')
dm_Assortment = pd.get_dummies(raw_store['Assortment'], prefix='Assortment')

df_store = df()
df_store['Store'] = raw_store['Store']
df_store[list(set(dm_StoreType))] = dm_StoreType
df_store[list(set(dm_Assortment))] = dm_Assortment

merge_table = pd.merge(df_train, df_store, on='Store')
merge_table.head()

,Store,Date,Sales,Promo,SchoolHoliday,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,DayOfWeek_4,DayOfWeek_3,...,StateHoliday_c,StateHoliday_0,StateHoliday_a,StoreType_c,StoreType_b,StoreType_a,StoreType_d,Assortment_a,Assortment_c,Assortment_b
0,1,2015-07-31,5263,1,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,2015-07-30,5020,1,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,2015-07-29,4782,1,1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1,2015-07-28,5011,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1,2015-07-27,6102,1,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


* * *  
## 0.3. 선행 학습(pilot modeling)
이제 1차적으로 현재까지의 데이터셋(`merge_table`)을 가지고 초벌 학습을 할꺼다.  
~~사실 지난과제에서 수행했어야 하는 부분...~~  


In [8]:
from datetime import datetime as dt
tmp_idx = list(merge_table.index.values)
dt_date = [dt.strptime(merge_table['Date'][idx], '%Y-%m-%d') for idx in tmp_idx]
merge_table['dt_date'] = pd.Series(dt_date, index=tmp_idx)

X_train = merge_table[merge_table['dt_date']<=dt(2013,12,31)]
X_test = merge_table[merge_table['dt_date']<=dt(2014,1,7)][merge_table['dt_date']>dt(2013,12,31)]
y_train = X_train['Sales']
y_test = X_test['Sales']
for key in ['Store', 'Date', 'Sales', 'dt_date'] :
    del X_train[key]
    del X_test[key]

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [9]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
# The coefficients
print('Coefficients: \n', mlr.coef_)
# mean square error
print("RSS: %.4f" % np.mean((mlr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % mlr.score(X_test, y_test))

Coefficients: 
 [  2.20866465e+03   1.86297859e+02  -5.18599906e+13  -5.18599906e+13
  -5.18599906e+13  -5.18599906e+13  -5.18599906e+13  -5.18599906e+13
  -5.18599906e+13  -2.91452587e+11  -2.91452587e+11  -2.91452587e+11
  -2.91452587e+11   3.27392817e+14   3.27392817e+14   3.27392817e+14
   3.27392817e+14   2.27114755e+14   2.27114755e+14   2.27114755e+14]
RSS: 9058306.3060
Variance score: 0.2413


* * *  
## 1. How to use CompetitionOpenSince[Month/Year]  

`Store` 테이블에 있는 `CompetitionOpenSince[Month/Year]` 변수를 다시 살펴봐야겠다.

In [11]:
raw_store[['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']].head()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear
0,1,1270.0,9.0,2008.0
1,2,570.0,11.0,2007.0
2,3,14130.0,12.0,2006.0
3,4,620.0,9.0,2009.0
4,5,29910.0,4.0,2015.0


왜 `float` 타입으로 되어 있는지 모르겠지만, 사실 별 타입에 대한 의미는 없다.  
그냥 몇년도 몇월달에 경쟁사가 생겼는지에 대한 정보이고, 경쟁사와의 거리도 유의미 할 것이라고 생각된다.  
왜냐면 수업시간에 교수님께서 언급했다싶이, 경쟁사의 업체가 상식적으로 너무 멀면 경쟁사가 없는것과 같기 때문이다.  
  
그래서 이걸 어떻게 활용할 수 있을까에 대해 고민해보았다.  
일단 시계열상에서 경쟁업체가 생긴 전(before)와 후(after)에 대한 차이를 구분해 줘야 겠다.  
1. `CompetitionOpenSinceMonth`, `CompetitionOpenSinceYear` 변수를 조합해 *`datetime`* 자료형을 만든다.
2. 현재 working table인 `merge_table`의 `Date` 변수와 비교해 경쟁사 유무여부를 *`binary`* 변수로 표현하고자 한다.

In [12]:
from math import isnan
def get_dt_CompetitionOpen(y, m) :
    if isnan(y) or isnan(m) :
        return dt(1900,1,1)
    else :
        return dt(int(y), int(m), 1)
df_store = raw_store[['Store', 'CompetitionDistance']]
tmp_idx = list(df_store.index.values)
dt_date = [get_dt_CompetitionOpen(raw_store['CompetitionOpenSinceYear'][idx], raw_store['CompetitionOpenSinceMonth'][idx]) for idx in tmp_idx]
df_store['dt_CompetitionOpen'] = pd.Series(dt_date, index=tmp_idx)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
def get_bin_CompetitionOpen(idx) :
    tmp_date = merge_table['dt_date'][idx]
    co_date = list(df_store[df_store['Store']==merge_table['Store'][idx]]['dt_CompetitionOpen'])[0]
    if tmp_date > co_date :
        return 1
    else :
        return 0
tmp_idx = list(merge_table.index.values)
bin_CO = [get_bin_CompetitionOpen(idx) for idx in tmp_idx]
merge_table['bin_CompetitionOpen'] = pd.Series(bin_CO, index=tmp_idx)

In [14]:
X_train = merge_table[merge_table['dt_date']<=dt(2013,12,31)]
X_test = merge_table[merge_table['dt_date']<=dt(2014,1,7)][merge_table['dt_date']>dt(2013,12,31)]
y_train = X_train['Sales']
y_test = X_test['Sales']
for key in ['Store', 'Date', 'Sales', 'dt_date'] :
    del X_train[key]
    del X_test[key]

mlr = LinearRegression()
mlr.fit(X_train, y_train)
# The coefficients
print('Coefficients: \n', mlr.coef_)
# mean square error
print("RSS: %.4f" % np.mean((mlr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % mlr.score(X_test, y_test))


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


Coefficients: 
 [  2.20751207e+03   1.87393106e+02  -5.06486575e+13  -5.06486575e+13
  -5.06486575e+13  -5.06486575e+13  -5.06486575e+13  -5.06486575e+13
  -5.06486575e+13  -4.36240302e+11  -4.36240302e+11  -4.36240302e+11
  -4.36240301e+11   3.08842070e+14   3.08842070e+14   3.08842070e+14
   3.08842070e+14   2.18379181e+14   2.18379181e+14   2.18379181e+14
  -5.95250000e+02]
RSS: 9072523.0242
Variance score: 0.2401


효과는 미미했다...

* * * 
## 2. How to use Promo2Since[Year/Week]

`Store` 테이블에 있는 `Promo2Since[Year/Week]` 변수를 다시 살펴봐야겠다.

In [15]:
raw_store[['Store', 'Promo2', 'Promo2SinceYear', 'Promo2SinceWeek']].head()

,Store,Promo2,Promo2SinceYear,Promo2SinceWeek
0,1,0,NaN,NaN
1,2,1,2010.0,13.0
2,3,1,2011.0,14.0
3,4,0,NaN,NaN
4,5,0,NaN,NaN


위의 방법과 동일하게 `binary` 변수를 만들어서 처리할 방법을 생각했었다.  
근데 이전변수에서 처리 결과가 미미한 걸 보고 별로 좋은 방법이 아니라는 생각을 했다...